In [1]:
import tensorflow as tf
import numpy as np
import gym
import datetime
from tqdm import tqdm
import random
from go_ai import data, metrics, mcts, models, policies
import matplotlib.pyplot as plt

# Hyperparameters

In [2]:
BOARD_SIZE = 7

In [3]:
ITERATIONS = 256
EPISODES_PER_ITERATION = 2 #128
MAX_STEPS = 2 * BOARD_SIZE**2
BATCH_SIZE = 32

In [4]:
NUM_EVAL_GAMES = 32
ITERATIONS_PER_EVAL = 1

In [5]:
LEARNING_RATE = 2e-3

In [6]:
MC_SIMS = 0
TEMP_FUNC = lambda step: (1/2) if (step < 16) else 0

In [7]:
WEIGHTS_DIR = 'model_weights/'
ACTOR_CRITIC_PATH = WEIGHTS_DIR + 'checkpoint_{}x{}.h5'.format(BOARD_SIZE, BOARD_SIZE)
LOAD_SAVED_MODELS = True

# Go Environment
Train on a small board with heuristic reward for fast training and efficient debugging

In [8]:
go_env = gym.make('gym_go:go-v0', size=BOARD_SIZE)

# Metrics and Tensorboard

In [9]:
!rm -rf ./logs/

Metrics

In [10]:
tb_metrics = {}
for metric_key in ['val_loss', 'overall_loss', 'num_steps', 'move_loss']:
    tb_metrics[metric_key] = tf.keras.metrics.Mean('{}'.format(metric_key), 
                                                   dtype=tf.float32)
tb_metrics['pred_win_acc'] = tf.keras.metrics.Accuracy()

Tensorboard

In [11]:
current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
log_dir = 'logs/actor_critic/{}/main'.format(current_time)
summary_writer = tf.summary.create_file_writer(log_dir)    

# Machine Learning Models

In [12]:
actor_critic = models.make_actor_critic(BOARD_SIZE, 'val_net', 'tanh')

Created layers
Created layers


In [13]:
_ = tf.keras.utils.plot_model(actor_critic, to_file='logs/model.png')

Failed to import pydot. You must install pydot and graphviz for `pydotprint` to work.
Failed to import pydot. You must install pydot and graphviz for `pydotprint` to work.


In [14]:
actor_critic.summary()

Model: "actor_critic"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
board (InputLayer)              [(None, 7, 7, 6)]    0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 7, 7, 64)     3520        board[0][0]                      
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 7, 7, 64)     256         conv2d[0][0]                     
__________________________________________________________________________________________________
re_lu (ReLU)                    (None, 7, 7, 64)     0           batch_normalization[0][0]        
_______________________________________________________________________________________

In [15]:
opponent = tf.keras.models.clone_model(actor_critic)

In [16]:
if LOAD_SAVED_MODELS:
    actor_critic.load_weights(ACTOR_CRITIC_PATH)
    opponent.load_weights(ACTOR_CRITIC_PATH)
    print("Loaded saved models")

Loaded saved models
Loaded saved models


In [17]:
state = go_env.get_state()
my_policy = policies.MctPolicy(actor_critic, state, MC_SIMS, TEMP_FUNC)
opponent_policy = policies.MctPolicy(opponent, state, MC_SIMS, TEMP_FUNC)
greedy_policy = policies.MctGreedyPolicy(state)
random_policy = policies.RandomPolicy()
human_policy = policies.HumanPolicy()

# Demo Trajectories

Symmetries

# Train

In [18]:
actor_critic_opt = tf.keras.optimizers.Adam(LEARNING_RATE)
replay_mem = []

In [19]:
data.make_episodes(BOARD_SIZE, ACTOR_CRITIC_PATH, EPISODES_PER_ITERATION, MAX_STEPS, outdir='./data/', num_workers=4)

Launching process 0
Launching process 0
Waiting on process 0
Waiting on process 0


In [ ]:
for iteration in range(ITERATIONS):
    # Train
    data.make_episodes(go_env, my_policy, EPISODES_PER_ITERATION, MAX_STEPS, out='data', num_workers=4)
    tb_metrics['num_steps'].update_state(num_steps)
        
    # Update the models (also shuffles memory)
    random.shuffle(replay_mem)
    np_data = data.replay_mem_to_numpy(replay_mem)
    batched_np_data = [np.array_split(datum, len(replay_mem) // BATCH_SIZE) 
                       for datum in np_data]
    batched_mem = list(zip(*batched_np_data))
    models.update_win_prediction(actor_critic, batched_mem, actor_critic_opt, 
                                 iteration, tb_metrics)
    
    # Evaluate against previous model
    if (iteration+1) % ITERATIONS_PER_EVAL == 0:
        rand_win_rate = metrics.evaluate(go_env, my_policy, random_policy,
                                                 max_steps=MAX_STEPS, 
                                                 num_games=8)
        greed_win_rate = metrics.evaluate(go_env, my_policy, greedy_policy,
                                                 max_steps=MAX_STEPS, 
                                                 num_games=8)
        opp_win_rate = metrics.evaluate(go_env, my_policy, opponent_policy, 
                                    max_steps=MAX_STEPS, 
                                    num_games=NUM_EVAL_GAMES)
        
        stats = "{:.1f}%O, {:.1f}%R, {:.1f}%G".format(100*opp_win_rate, 
                                                     100*rand_win_rate,
                                                     100*greed_win_rate)
        if opp_win_rate > 0.6:
            actor_critic.save_weights(ACTOR_CRITIC_PATH)
            opponent.load_weights(ACTOR_CRITIC_PATH)
            
            print("{} Accepted new model".format(stats))
        else:
            print("{} Rejected new model".format(stats))
            actor_critic.load_weights(ACTOR_CRITIC_PATH)
    
    # Log results and resets the metrics
    
    metrics.log_to_tensorboard(summary_writer, tb_metrics, iteration, go_env, 
                               actor_critic, TEMP_FUNC, 'logs/a_trajectory.jpg')
    # Reset memory
    replay_mem.clear()

# Evaluate

Play against our AI

In [ ]:
go_env = gym.make('gym_go:go-v0', size=BOARD_SIZE)
data.pit(go_env, black_policy=opponent_policy, white_policy=human_policy, max_steps=MAX_STEPS)